---
title: "Data Cleaning"
format:
    html: 
        code-fold: false
---

<!-- After digesting the instructions, you can delete this cell, these are assignment instructions and do not need to be included in your final submission.  -->

{{< include instructions.qmd >}} 

# Code 

source code used for this section of the project here.(cumulative for previous section)
for histogram, boxplot after data cleaning process,
it will also be placed in next section: EDA


In [ ]:
#package
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
#using statcast data , by python api warpper pybaseball
from pybaseball import statcast
#get baseball data for 2024 mlb season(start from last season willing to add up to 3 more season)
# 2022 mlb season started in 4/7/2022 
#2024 mlb season ended in 11/2/2024
#including only regular season
data = statcast(start_dt = '2024-03-20',end_dt = '2024-11-02')
#getting possbible features
baseball_data  = data[['pitch_type','release_speed','release_pos_x', 'release_pos_z' ,
                       'pitcher', 'description','p_throws', "pfx_x","pfx_z",
                        'release_extension','release_spin_rate',
            'events',"game_type","release_pos_y",]]
#finding all events(play result)
len(baseball_data)
baseball_data["events"].unique() # there is a missing value (nan)
baseball_data.dtypes
# data cleaning
pitching  = baseball_data
#removing all na in events, because na in event means it is in ball counts, not the result of play
#which is not my interest
pitching = pitching[pitching['events'].notna()]

#only counts regular season :R represents regular season
pitching = pitching[pitching['game_type'] == "R"]


# if values in events contains following value, it is out:
#strikeout,field_out,sac_fly,force_out,grounded_into_double_play,double_play,
#strikeout_dpubleplay,
outs = ['strikeout','field_out','sac_fly','force_out','grounded_into_double_play','double_play',
        'strikeout_dpubleplay']

# if events has value strikeout or strikeouyt double play it will be assigned as 1, else 0
so = ["strikeout","strikeout_double_play"]

# assigning strikeout as 1 in newly defined column so
pitching['events'] = pitching['events'].fillna('').astype(str)
pitching['so'] = pitching['events'].apply(
    lambda x: 1 if any(event in so for event in x.split(', ')) else 0
)
#assigning outs as 1 in newly defiend column outs
pitching['outs'] = pitching['events'].apply(
    lambda x: 1 if any(event in outs for event in x.split(', ')) else 0
)
#checking data type and convert release spin rate to float 
pitching.dtypes
pitching['release_spin_rate'] = pitching['release_spin_rate'].astype(float)
#histogram before data cleaning(release speed)

pitching['release_speed'].hist(bins=30, edgecolor='k', figsize=(10, 6),color  = 'orange')
plt.title('Histogram of release speed(before cleaning process)',fontsize = 15)
plt.xlabel('release speed(MPH)',fontsize = 12)
plt.ylabel('Frequency',fontsize = 12)
plt.show()
sns.boxplot(pitching['release_speed'],color  = 'skyblue')
plt.title('Boxplot of release speed(before cleaning process)',fontsize = 15)
plt.ylabel('relase speed(before cleaning process)',fontsize = 12)
plt.show()
#from the plot,need to remove outliers 
# so use IQR method 
release_speed1 = pitching["release_speed"].quantile(0.25)
release_speed3 = pitching["release_speed"].quantile(0.75)
iqr1 = release_speed3-release_speed1
low1 = release_speed1-1.5*iqr1
upper1 = release_speed3+1.5*iqr1
pitching_data = pitching[(pitching["release_speed"] >= low1) & (pitching["release_speed"] <= upper1)]
#histogram before data cleaning(release spin rate)
pitching_data['release_spin_rate'].hist(bins=30, edgecolor='k', figsize=(8, 5),color  = 'orange')
plt.title('Histogram of release spin rate (before cleaning process)',fontsize = 15)
plt.xlabel('release spin rate ',fontsize = 12)
plt.ylabel('Frequency',fontsize = 12)
plt.show()
#boxplot for detecting outliers
sns.boxplot(pitching_data['release_spin_rate'],color  = 'skyblue')
plt.title('Boxplot of release spin rate(before cleaning process)',fontsize = 15)
plt.ylabel('relase spin rate(before cleaning process)',fontsize = 12)
plt.show()
#based on the description statistics, spin_rate also has outlier and missing value 
#from the plot it is cetain that there is a outlier
#using same method 
release_spin_rate1 = pitching_data["release_spin_rate"].quantile(0.25)
release_spin_rate3 = pitching_data["release_spin_rate"].quantile(0.75)

iqr2 = release_spin_rate3-release_spin_rate1
low2 = release_spin_rate1-1.5*iqr2
upper2 = release_spin_rate3+1.5*iqr2

pitching_data = pitching_data[(pitching_data["release_spin_rate"] >= low2) & (pitching_data["release_spin_rate"] <= upper2)]
#graph for release extension 
pitching_data['release_extension'].hist(bins=30, edgecolor='k', figsize=(8, 5),color  = "orange")
plt.title('Histogram of release extension(before cleaning)',fontsize = 15)
plt.xlabel('release extension',fontsize = 12)
plt.ylabel('Frequency',fontsize = 12)
plt.show()
sns.boxplot(pitching_data['release_extension'],color  = "skyblue")
plt.title('Boxplot of release extension(before cleaning)',fontsize = 15)
plt.ylabel('release extension(before cleaning)',fontsize = 12)
plt.show()
# there are NA and outliers, use IQR to remove these

release_extension1 = pitching_data["release_extension"].quantile(0.25)
release_extension3 = pitching_data["release_extension"].quantile(0.75)

iqr3 = release_extension3-release_extension1

low3 = release_extension1-1.5*iqr3
upper3 = release_extension3+1.5*iqr3
pitching_data = pitching_data[(pitching_data["release_extension"] >= low3) & (pitching_data["release_extension"] <= upper3)]
#check again for na values: no more na in float features, move on to categorical features
print(pitching_data.isnull().sum())
# now it is cleaned
#handing cateogrical variables
baseball_data.dtypes
#there are five object types: pitch_type,description,p_throws,events,game_type

#checking pitch types
pitch_type_counts = pitching_data['pitch_type'].value_counts()
# show count of pitch types
print(pitch_type_counts)
#for pitch type, only ones that are listed in MLB.com will be used 
#https://www.mlb.com/glossary/pitch-types/four-seam-fastball
#also for clear description, change pitch type
#sv,sc,fa,po,cs has less than 1000 rows so it will be removed
pitching_data= pitching_data[pitching_data['pitch_type'].apply(lambda x: x not in ['CS', 'FA','PO','SC','SV'])]

# create column called ball type
# which defines whether the pitch was fastball or breaking ball
# from the data, three values FF(four seam fastball),FC(cutter) will be assigned as fastball
# these are based on MLB official website classification of pitch types
#if it was a fastball assign 1 else 0
fastballs = ["FF","FC"]
pitching_data['pitch_type'] = pitching_data['pitch_type'].fillna('').astype(str)
pitching_data['ball_types'] = pitching_data['pitch_type'].apply(
    lambda x: 1 if any(pitch_type in fastballs for pitch_type in x.split(', ')) else 0
)
#categorical variable
#use one hot coding for cateogorical variable: pitch type and p_throws, which will be used for 
#later analysis
pitching_dummies = pd.get_dummies(pitching_data[["pitch_type","p_throws"]],dtype= int,prefix= 'pitch')
pitching_data = pd.concat([pitching_data,pitching_dummies],axis = 1)
# for pitch type, only ones that are listed in MLB.com will be used 
#also for clear description, change pitch type and pitch throws(lefty or righty)related column names
print(pitching_dummies.columns.tolist())
pitching_data= pitching_data.rename(columns={'pitch_CH': 'changeup',
                                             'pitch_CU':'curveball',
                                            'pitch_FC':'cutter',
                                            'pitch_FF' : 'four-seam_fastball',
                                            'pitch_KN': 'knuckleball',
                                            'pitch_SI':'sinker',
                                            'pitch_SL':'slider',
                                            'pitch_SV':'slurve',
                                            'pitch_FS':'splitter',
                                            'pitch_ST':'sweeper',
                                            'pitch_KC':'knuckle-curve',
                                            'pitch_L':'lefty',
                                            'pitch_R':'righty'})
pitch_label = ['changeup','curveball','cutter','four-seam_fastball', 'knuckleball','sinker','slider','slurve',
'splitter','pitch_ST','sweeper','pitch_KC','knuckle-curve']